In [225]:
# http://ec2-54-159-219-64.compute-1.amazonaws.com:8888/notebooks/edmunds_spark_dataframe.ipynb#

# Edmunds Spark DataFrames

In [152]:
# !sudo pip install pandas

from pyspark.sql.functions import desc, explode
from pprint import pprint

In [153]:
# A way to read json files:
df_0 = spark.read.load("s3a://edmundsvehicle/2017/03/06/21/*", format="json")

# Create 4 tables that conform to third normal form:
### MODEL, YEARS, STYLES, SUBMODELS

In [199]:
# first work with a single json object
MODEL = spark.read.json("s3a://edmundsvehicle/2017/03/06/21/*")

In [155]:
MODEL.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- niceName: string (nullable = true)
 |-- time_stamp: double (nullable = true)
 |-- years: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- styles: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- submodel: struct (nullable = true)
 |    |    |    |    |    |-- body: string (nullable = true)
 |    |    |    |    |    |-- modelName: string (nullable = true)
 |    |    |    |    |    |-- niceName: string (nullable = true)
 |    |    |    |    |-- trim: string (nullable = true)
 |    |    |-- year: long (nullable = true)



In [156]:
# Create First DataFrame.
# Let's test first our query.

# years[0] 
MODEL.selectExpr('id').show()

+------+
|    id|
+------+
|BMW_i3|
+------+



In [157]:
YEARS = MODEL.select(MODEL['id'], explode(MODEL['years']))
YEARS.show()

+------+--------------------+
|    id|                 col|
+------+--------------------+
|BMW_i3|[200460080,Wrappe...|
|BMW_i3|[200725667,Wrappe...|
|BMW_i3|[401612494,Wrappe...|
|BMW_i3|[401690178,Wrappe...|
+------+--------------------+



In [158]:
YEARS.printSchema()

root
 |-- id: string (nullable = true)
 |-- col: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- styles: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- submodel: struct (nullable = true)
 |    |    |    |    |-- body: string (nullable = true)
 |    |    |    |    |-- modelName: string (nullable = true)
 |    |    |    |    |-- niceName: string (nullable = true)
 |    |    |    |-- trim: string (nullable = true)
 |    |-- year: long (nullable = true)



In [159]:
YEARS = YEARS.withColumn("year_id", YEARS['col'].getField("id"))
YEARS = YEARS.withColumn("year", YEARS['col'].getField("year"))
YEARS = YEARS.withColumn("styles", YEARS['col'].getField("styles"))
YEARS.show()

+------+--------------------+---------+----+--------------------+
|    id|                 col|  year_id|year|              styles|
+------+--------------------+---------+----+--------------------+
|BMW_i3|[200460080,Wrappe...|200460080|2014|[[101402984,4dr H...|
|BMW_i3|[200725667,Wrappe...|200725667|2015|[[200725669,4dr H...|
|BMW_i3|[401612494,Wrappe...|401612494|2016|[[401612496,4dr H...|
|BMW_i3|[401690178,Wrappe...|401690178|2017|[[401690179,4dr H...|
+------+--------------------+---------+----+--------------------+



In [161]:
STYLES = YEARS.select(YEARS['id'], explode(YEARS['styles']))
STYLES.show()

+------+--------------------+
|    id|                 col|
+------+--------------------+
|BMW_i3|[101402984,4dr Ha...|
|BMW_i3|[200689763,4dr Ha...|
|BMW_i3|[200725669,4dr Ha...|
|BMW_i3|[200725668,4dr Ha...|
|BMW_i3|[401612496,4dr Ha...|
|BMW_i3|[401612495,4dr Ha...|
|BMW_i3|[401690179,4dr Ha...|
|BMW_i3|[401690196,4dr Ha...|
|BMW_i3|[401690180,60 Ah ...|
+------+--------------------+



In [162]:
STYLES.printSchema()

root
 |-- id: string (nullable = true)
 |-- col: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- submodel: struct (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- modelName: string (nullable = true)
 |    |    |-- niceName: string (nullable = true)
 |    |-- trim: string (nullable = true)



In [166]:
STYLES = STYLES.withColumn("trim_id", STYLES['col'].getField("id"))
STYLES = STYLES.withColumn("name", STYLES['col'].getField("name"))
STYLES = STYLES.withColumn("submodel", STYLES['col'].getField("submodel"))
STYLES = STYLES.withColumn("trim", STYLES['col'].getField("trim"))
STYLES.show()

+------+--------------------+---------+--------------------+--------------------+----------------+
|    id|                 col|  trim_id|                name|            submodel|            trim|
+------+--------------------+---------+--------------------+--------------------+----------------+
|BMW_i3|[101402984,4dr Ha...|101402984|4dr Hatchback w/R...|[Hatchback,i3 Hat...|w/Range Extender|
|BMW_i3|[200689763,4dr Ha...|200689763|4dr Hatchback (el...|[Hatchback,i3 Hat...|            Base|
|BMW_i3|[200725669,4dr Ha...|200725669|4dr Hatchback (el...|[Hatchback,i3 Hat...|            Base|
|BMW_i3|[200725668,4dr Ha...|200725668|4dr Hatchback w/R...|[Hatchback,i3 Hat...|w/Range Extender|
|BMW_i3|[401612496,4dr Ha...|401612496|4dr Hatchback w/R...|[Hatchback,i3 Hat...|w/Range Extender|
|BMW_i3|[401612495,4dr Ha...|401612495|4dr Hatchback (el...|[Hatchback,i3 Hat...|            Base|
|BMW_i3|[401690179,4dr Ha...|401690179|4dr Hatchback w/R...|[Hatchback,i3 Hat...|w/Range Extender|
|BMW_i3|[4

In [180]:
SUBMODELS = STYLES.select(STYLES['id'], STYLES.submodel.body, STYLES.submodel.modelName, STYLES.submodel.niceName)
SUBMODELS.show()

+------+-------------+------------------+-----------------+
|    id|submodel.body|submodel.modelName|submodel.niceName|
+------+-------------+------------------+-----------------+
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
|BMW_i3|    Hatchback|      i3 Hatchback|        hatchback|
+------+-------------+------------------+-----------------+



In [183]:
# run a SQL query

sqlCtx.registerDataFrameAsTable(MODEL, "MODEL")
sqlCtx.registerDataFrameAsTable(YEARS, "YEARS")
sqlCtx.registerDataFrameAsTable(STYLES, "STYLES")
sqlCtx.registerDataFrameAsTable(SUBMODELS, "SUBMODELS")

In [198]:
df2 = sqlCtx.sql("""SELECT m.id, y.year from MODEL m
                    LEFT JOIN YEARS y on m.id = y.id""")
df2.collect()
df2.show()

+------+----+
|    id|year|
+------+----+
|BMW_i3|2017|
|BMW_i3|2016|
|BMW_i3|2015|
|BMW_i3|2014|
+------+----+



# Create Tables Using All the Models in the dataset

In [205]:
# new_df = df.select(df['id'], explode(df['years']['years']))

In [217]:
# We read all files:
ALL_MODELS = spark.read.json("s3a://edmundsvehicle/2017/*/*/*/*")

In [218]:
ALL_MODELS.cache()
type(ALL_MODELS)

pyspark.sql.dataframe.DataFrame

In [219]:
# df_all.selectExpr('id',
#                   'niceName',
#                   'time_stamp',
#                   'years[0].id as model_year_id',
#                   'years[0].styles[0].id as style_id',
#                   'years[0].styles[0].name as style',
#                   'years[0].styles[0].submodel.body as body',
#                   'years[0].styles[0].submodel.modelName as modelName',
#                   'years[0].styles[0].trim as trim').show(500)
# df_all.selectExpr('years').show(500)

In [224]:
# convert all to 3rd normal form schema

ALL_YEARS = ALL_MODELS.select(ALL_MODELS['id'], explode(ALL_MODELS['years']))

ALL_YEARS = ALL_YEARS.withColumn("year_id", ALL_YEARS['col'].getField("id"))
ALL_YEARS = ALL_YEARS.withColumn("year", ALL_YEARS['col'].getField("year"))
ALL_YEARS = ALL_YEARS.withColumn("styles", ALL_YEARS['col'].getField("styles"))

ALL_STYLES = ALL_YEARS.select(ALL_YEARS['id'], explode(ALL_YEARS['styles']))

ALL_STYLES = ALL_STYLES.withColumn("trim_id", ALL_STYLES['col'].getField("id"))
ALL_STYLES = ALL_STYLES.withColumn("name", ALL_STYLES['col'].getField("name"))
ALL_STYLES = ALL_STYLES.withColumn("submodel", ALL_STYLES['col'].getField("submodel"))
ALL_STYLES = ALL_STYLES.withColumn("trim", ALL_STYLES['col'].getField("trim"))

ALL_SUBMODELS = ALL_STYLES.select(ALL_STYLES['id'], ALL_STYLES.submodel.body, ALL_STYLES.submodel.modelName, ALL_STYLES.submodel.niceName)

# register as SQL table
sqlCtx.registerDataFrameAsTable(ALL_MODELS, "ALL_MODELS")
sqlCtx.registerDataFrameAsTable(ALL_YEARS, "ALL_YEARS")
sqlCtx.registerDataFrameAsTable(ALL_STYLES, "ALL_STYLES")
sqlCtx.registerDataFrameAsTable(ALL_SUBMODELS, "ALL_SUBMODELS")

# run a SQL query
test_query = sqlCtx.sql("""SELECT m.id, y.year from ALL_MODELS m
                    LEFT JOIN ALL_YEARS y on m.id = y.id
                    WHERE m.id is not NULL
                    LIMIT 5""")
test_query.collect()
test_query.toPandas()

,id,year
0,BMW_3_Series,2017
1,BMW_3_Series,2016
2,BMW_3_Series,2015
3,BMW_3_Series,2014
4,BMW_3_Series,2013


In [223]:
# df_all.toPandas()